#### Tensorflow 6강. File load in Tensorflow and Queue Thread
##### https://www.youtube.com/watch?v=hPkmxczEj6k&index=6&list=PL1H8jIvbSo1qOtjQXFzBxMWjL_Gc5x3yG&t=940s

In [1]:
import tensorflow as tf
import os
from PIL import Image

In [2]:
image_dir=os.getcwd()+"/images.jpeg"
label_dir=os.getcwd()+"/label.csv"

imagenmae_list=[image_dir]
labelnmae_list=[label_dir]

imagenmae_queue = tf.train.string_input_producer(imagenmae_list)
labelnmae_queue = tf.train.string_input_producer(labelnmae_list)

image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()

image_key, image_value = image_reader.read(imagenmae_queue)
label_key, label_value = label_reader.read(labelnmae_queue)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)    
    print(sess.run(image_key))
    print(sess.run(label_key))    
    coord.request_stop()
    coord.join(threads)


image =tf.image.decode_png(image_value)
label =tf.decode_csv(label_value,record_defaults= [[0]])

x= tf.cast(image, tf.float32)
y_ = tf.cast(label, tf.float32)
y_ = tf.reshape(y_,[-1,1])

b'C:\\kimhk\\workspace\\program\\Python\\IPNB/images.jpeg'
b'C:\\kimhk\\workspace\\program\\Python\\IPNB/label.csv:1'


In [14]:
print(y_)
print(tf.__version__)

Tensor("Reshape:0", shape=(1, 1), dtype=float32)
0.12.1


In [4]:
image_width = 49
image_hight = 61

In [5]:
#x = tf.placeholder(tf.float32,shape=[None,image_width,image_hight ])
#y_ = tf.placeholder(tf.float32,shape=[None,1]) # 정답데이터가 몇 차원

In [6]:
# hidden1 convolution
hidden1_w = tf.Variable(tf.truncated_normal([5,5,1,32]))
hidden1_b = tf.Variable(tf.zeros([32]))

x_image = tf.reshape(x, [-1, image_width, image_hight,1])

h1=tf.nn.relu(tf.nn.conv2d(x_image, hidden1_w, strides=[1,1,1,1], padding="SAME" ) +hidden1_b)


In [7]:
# hidden2 convolution
hidden2_w = tf.Variable(tf.truncated_normal([5,5,32,64]))
hidden2_b = tf.Variable(tf.zeros([64]))

h2=tf.nn.relu(tf.nn.conv2d(h1, hidden2_w, strides=[1,1,1,1], padding="SAME" ) + hidden2_b)


In [8]:
h_flat = tf.reshape(h2, [-1, 49*61*64])
fc_w = tf.Variable(tf.truncated_normal([49*61*64,10]))
fc_b = tf.Variable(tf.zeros([10]))

h_fc1= tf.nn.relu(tf.matmul(h_flat, fc_w) + fc_b)

In [9]:
print(h_flat)
print(fc_w)
print(fc_b)
print(h_fc1)

Tensor("Reshape_2:0", shape=(?, 191296), dtype=float32)
Tensor("Variable_4/read:0", shape=(191296, 10), dtype=float32)
Tensor("Variable_5/read:0", shape=(10,), dtype=float32)
Tensor("Relu_2:0", shape=(?, 10), dtype=float32)


In [10]:
out_w = tf.Variable(tf.truncated_normal([10,1]))
out_b = tf.Variable(tf.zeros([1]))

pred = tf.matmul(h_fc1, out_w) + out_b

In [11]:
#cross_entropy 
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(pred, y_))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [12]:
#[T,T,F,F,F,F,T] BOOL
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)    
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        sess.run(train)   
        cost, acc =  sess.run([loss, accuracy])   
        print("loss:",cost)  
        print("acc:",acc)       
            
    #image = sess.run(image)
    #print(image)
    #Image.fromarray(image).show()
    
    coord.request_stop()
    coord.join(threads)

loss: 4.55189e+06
acc: 1.0
loss: 2.93801e+06
acc: 1.0
loss: 1.31773e+06
acc: 1.0
loss: -274039.0
acc: 1.0
loss: -1.72929e+06
acc: 1.0
loss: -3.1803e+06
acc: 1.0
loss: -4.62626e+06
acc: 1.0
loss: -6.06903e+06
acc: 1.0
loss: -7.30265e+06
acc: 1.0
loss: -8.41133e+06
acc: 1.0
